In [1]:
import tensorflow as tf
import numpy as np
import os

%load_ext autoreload
%autoreload 2

# Import the library to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
FOLDERNAME = 'DL project/fma'
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1Gzbs2MQ8ttHuTOpxDaZoGkbdHOw1ukQO/DL project/fma


In [6]:
# import os
# import numpy as np

# dataset_dir = "data/spec_small2/"  # Update with your dataset directory

# def check_npy_files(dataset_dir):
#     for class_name in sorted(os.listdir(dataset_dir)):
#         class_path = os.path.join(dataset_dir, class_name)
#         if not os.path.isdir(class_path):
#             continue
#         for file_name in os.listdir(class_path):
#             if file_name.endswith(".npy"):
#                 file_path = os.path.join(class_path, file_name)
#                 try:
#                     data = np.load(file_path)  # Try loading the file
#                     if data.size == 0:  # Check if the loaded data is empty
#                         print(f"WARNING: Empty .npy file found: {file_path}")
#                 except EOFError:
#                     print(f"WARNING: Corrupted .npy file found: {file_path}")
#                 except Exception as e:
#                     print(f"WARNING: Error loading .npy file: {file_path}, Error: {e}")

# # Call the function to check your files
# check_npy_files(dataset_dir + "train")
# check_npy_files(dataset_dir + "val")

# i = 0
# f = set()
# for subdir, dirs, files in os.walk('data/spec_small2/'):
#     for file in files:
#         i += 1
# print(i)
        # if '(' in file:
        #     file_path = os.path.join(subdir, file)
        #     print(file)
        #     os.remove(file_path)
        # num = int(file[:-4])
        # if num >= 139775 and num <= 141566:
        #     file_path = os.path.join(subdir, file)
        #     if os.path.exists(file_path):
        #         # os.remove(file_path)
        #         print(num, end = " ")


    #   npy = np.load(file_path)

    #   if npy.shape != (10000,):
    #     print(npy.shape)
    #     os.remove(file_path)
    #   i+= 1

    #   if i %100 == 0:
    #     print(i , file)

In [10]:
import math
from tensorflow import keras
from tensorflow.keras import layers, regularizers

NUM_CLASSES = 8

kaiming_normal = keras.initializers.VarianceScaling(scale=2.0, mode='fan_out', distribution='untruncated_normal')

def conv3x3(x, out_planes, stride=1, name=None):
    x = layers.ZeroPadding2D(padding=1, name=f'{name}_pad')(x)
    return layers.Conv2D(filters=out_planes, kernel_size=3, strides=stride, use_bias=False, kernel_initializer=kaiming_normal, name=name, kernel_regularizer=regularizers.L2(0.01),)(x)

def basic_block(x, planes, stride=1, downsample=None, name=None):
    identity = x

    out = conv3x3(x, planes, stride=stride, name=f'{name}.conv1')
    out = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name=f'{name}.bn1')(out)
    out = layers.ReLU(name=f'{name}.relu1')(out)

    out = conv3x3(out, planes, name=f'{name}.conv2')
    out = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name=f'{name}.bn2')(out)

    if downsample is not None:
        for layer in downsample:
            identity = layer(identity)

    out = layers.Add(name=f'{name}.add')([identity, out])
    out = layers.ReLU(name=f'{name}.relu2')(out)

    return out

def make_layer(x, planes, blocks, stride=1, name=None):
    downsample = None
    inplanes = x.shape[2]
    if stride != 1 or inplanes != planes:
        downsample = [
            layers.Conv2D(filters=planes, kernel_size=1, strides=stride, use_bias=False, kernel_initializer=kaiming_normal, name=f'{name}.0.downsample.0'),
            layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name=f'{name}.0.downsample.1'),
        ]

    x = basic_block(x, planes, stride, downsample, name=f'{name}.0')
    for i in range(1, blocks):
        x = basic_block(x, planes, name=f'{name}.{i}')

    return x

def resnet(x, blocks_per_layer, num_classes=1000):
    x = layers.Reshape((x.shape[-2],x.shape[-1], 1))(x)
    x = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name='bn0')(x)
    # x = layers.ZeroPadding2D(padding=3, name='conv1_pad')(x)
    x = layers.Conv2D(filters=64, kernel_size=7, strides=2, use_bias=False, kernel_initializer=kaiming_normal, name='conv1')(x)
    x = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name='bn1')(x)
    x = layers.ReLU(name='relu1')(x)
    # x = layers.ZeroPadding2D(padding=1, name='maxpool_pad')(x)
    x = layers.MaxPool2D(pool_size=3, strides=2, name='maxpool')(x)

    x = make_layer(x, 64, blocks_per_layer[0], name='layer1')
    x = make_layer(x, 128, blocks_per_layer[1], stride=2, name='layer2')
    x = make_layer(x, 256, blocks_per_layer[2], stride=2, name='layer3')
    x = make_layer(x, 512, blocks_per_layer[3], stride=2, name='layer4')

    x = layers.GlobalAveragePooling2D(name='avgpool')(x)
    initializer = keras.initializers.RandomUniform(-1.0 / math.sqrt(512), 1.0 / math.sqrt(512))
    x = layers.Dense(units=num_classes, kernel_initializer=initializer, bias_initializer=initializer, name='fc')(x)
    x = layers.Activation('softmax')(x)

    return x

def resnet18(x, **kwargs):
    return resnet(x,  [3, 4, 6, 3], **kwargs)

inputs = keras.Input(shape=(128, 12))
# inputs = keras.Input(shape=(59953,))
outputs = resnet18(inputs, num_classes=NUM_CLASSES)

model = keras.Model(inputs, outputs)



# model = tf.keras.models.Sequential([
#     layers.Reshape((128, 12, 1)),
#     tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 12, 1)), # Convolutional layer
#     tf.keras.layers.BatchNormalization(), # Batch normalization
#     tf.keras.layers.MaxPooling2D((2, 2)), # Max pooling
#     tf.keras.layers.Conv2D(64, (3, 3), activation='relu'), # Another convolutional layer
#     tf.keras.layers.BatchNormalization(), # Batch normalization
#     tf.keras.layers.MaxPooling2D((2, 2)), # Max pooling
#     tf.keras.layers.Flatten(), # Flatten the output for the dense layers
#     tf.keras.layers.Dense(8, activation='softmax') # Output layer with 10 units (e.g., for 10 classes)
# ])



In [12]:
import tensorflow as tf
import numpy as np
import os


dataset_dir = "data/spec_small2/"

# Function to load a single .npy file and assign its label
def load_npy_file(file_path, label):
    # Read the file and decode its path
    npy = tf.numpy_function(lambda path: np.load(path).astype(np.float32), [file_path], tf.float32)
    npy.set_shape([128,12])  # Set shape (update according to your spectrogram's dimensions)

    return npy, label


# Function to create a dataset for all files
def create_dataset(dataset_dir):
    # List all files and infer labels from folder names
    all_files = []
    all_labels = []
    for class_name in sorted(os.listdir(dataset_dir)):
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        label = ([int(digit) for digit in class_name])  # Convert folder name to integer label
        files = [os.path.join(class_path, f) for f in os.listdir(class_path) if f.endswith(".npy")]
        all_files.extend(files)
        all_labels.extend([label] * len(files))

    print(len(all_labels))
    print(len(all_files))



    # Create a dataset from the files and labels
    file_paths = tf.constant(all_files)
    labels = tf.constant(all_labels, dtype=tf.int32)
    num_classes = 8  # Determine the number of classes
    # labels = tf.one_hot(labels, depth=num_classes)  # Apply one-hot encoding
    # print(labels)

    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    dataset = dataset.shuffle(len(all_files))  # Shuffle dataset
    dataset = dataset.map(load_npy_file, num_parallel_calls=tf.data.AUTOTUNE)  # Load files
    batch_size = 64
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset, all_labels


# Create the train dataset
train_dataset, _ = create_dataset(dataset_dir+"train")
val_dataset, _ = create_dataset(dataset_dir+"val")
test_dataset, test_labels = create_dataset(dataset_dir+"test")


# Inspect a sample
# for spectrogram, label in train_dataset.take(1):
#     print("Spectrogram shape:", spectrogram.shape)
#     print("Label:", label.shape)
#     num_classes = label.shape[1]


6397
6397
800
800
800
800


In [16]:
from tensorflow.keras.callbacks import ModelCheckpoint

new_optimizer = keras.optimizers.Adam(learning_rate=0.00001)
model.compile(
    optimizer=new_optimizer ,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
checkpoint = ModelCheckpoint(
    filepath='best_small_specresnet_model30.keras',  # Filepath to save the model
    monitor='val_loss',       # Metric to monitor (e.g., 'val_loss', 'val_accuracy')
    save_best_only=True,      # Save only the best model
    save_weights_only=False,  # Save the entire model (not just weights)
    mode='min',               # Save when the monitored metric decreases ('min') or increases ('max')
    verbose=1                 # Print a message when saving
)




history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    verbose=1,
    callbacks=[checkpoint]
    # callbacks=[PrintEveryFewBatchesCallback(interval=1), checkpoint]
)

Epoch 1/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step - accuracy: 0.6320 - loss: 85.1470
Epoch 1: val_loss improved from inf to 83.38602, saving model to best_small_specresnet_model30.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 109s 388ms/step - accuracy: 0.6319 - loss: 85.1388 - val_accuracy: 0.2962 - val_loss: 83.3860
Epoch 2/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - accuracy: 0.6434 - loss: 81.7878
Epoch 2: val_loss improved from 83.38602 to 80.32890, saving model to best_small_specresnet_model30.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 55s 232ms/step - accuracy: 0.6435 - loss: 81.7802 - val_accuracy: 0.2837 - val_loss: 80.3289
Epoch 3/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - accuracy: 0.6855 - loss: 78.6690
Epoch 3: val_loss improved from 80.32890 to 77.50587, saving model to best_small_specresnet_model30.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 41s 230ms/step - accuracy: 0.6854 - loss: 78.6620 - val_accuracy: 0.3000 - val_loss: 77.5059
Epoch 4/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step

KeyboardInterrupt: 

In [18]:
model = tf.keras.models.load_model('final_models/best_small_specresnet_model.keras')
raw_pred = model.predict(test_dataset)
pred_labels = np.argmax(raw_pred, axis=1)
true_labels = np.argmax(test_labels, axis=1)

print(np.sum(pred_labels == true_labels)/800)
correct = [0]*8
for i in range(800):
    if pred_labels[i] == true_labels[i]:
        correct[true_labels[i]] +=1


print([c/100 for c in correct])

13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 309ms/step
0.14
[0.09, 0.17, 0.13, 0.2, 0.03, 0.2, 0.1, 0.2]
